<a href="https://colab.research.google.com/github/mkm-world/laduma-analytics/blob/main/zindi_football_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Zindi Weekendz: Laduma Analytics Football League Tables

- A little notebok for showing the table/standings at the closing of every day.

## Import Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import itertools
from tqdm import tqdm
%matplotlib inline
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from datetime import date

# Downloading all datasets directly from Zindi platform using url:

In [ ]:
import requests, zipfile
# The url and auth_value from the website
myobj = {'auth_token': } # input your auth token here 
data_list=['Test.csv','VariableDefinitions.csv','Train.csv','SampleSubmission.csv','extra_data.csv','test_game_statistics.csv','train_game_statistics.csv']
target_dir=''
base_path='https://api.zindi.africa/v1/competitions/laduma-analytics-football-league-winners-prediction-challenge/files/'
def load_zindi_data(data_list,base_path,target_dir):
  for data in data_list:
      target_path=  target_dir +data
      data_path=base_path+ data
      x = requests.post(data_path, data = myobj,stream=True)
      handle = open(target_path, "wb")
      for chunk in x.iter_content(chunk_size=512):
        if chunk:  # filter out keep-alive new chunks
          handle.write(chunk)
      handle.close()
load_zindi_data(data_list,base_path,target_dir)

#Load Datasets:

In [ ]:
train = pd.read_csv("Train.csv",parse_dates=['Date'])
test = pd.read_csv("Test.csv",parse_dates=['Date'])
train_game_statistics = pd.read_csv("/content/train_game_statistics.csv")
test_game_statistics = pd.read_csv("/content/test_game_statistics.csv")

## Little Feature Engineering 

In [ ]:
all_data['Team']=all_data['Home Team']
all_data.loc[all_data['Score']=='Draw','draw']=1
cop_all_data=all_data.copy()
all_data.loc[all_data['Score']=='Home Win','win']=1
all_data.loc[all_data['Score']=='Away win','lose']=1
all_data['side']='Home'
cop_all_data.loc[cop_all_data['Score']=='Home Win','lose']=1
cop_all_data.loc[cop_all_data['Score']=='Away win','win']=1
cop_all_data['Team']=all_data['Away Team']
cop_all_data['side']='Away'
all_data=pd.concat([all_data,cop_all_data]).reset_index(drop=True)
all_data.fillna(0,inplace=True)

In [ ]:
goals_stats=train_game_statistics.groupby(['Game_ID','Team'])['Goals_scored','Goals_conceded'].sum().reset_index()
all_data=pd.merge(goals_stats,all_data,on=['Game_ID','Team'])

In [ ]:
def show_table(current_date):
  prev_data=all_data.loc[all_data.Date.dt.date<=current_date]
  if prev_data.shape[0]==0:
    print('data not given,pick a later date')
  else :
    season=prev_data.Season.max()
    season_data=prev_data.loc[prev_data['Season']==season]
    table=season_data.groupby(['Team']).win.sum().reset_index()
    table['Played']=season_data.groupby(['Team']).win.count().values
    table['Draw']=season_data.groupby(['Team']).draw.sum().values
    table['Loss']=season_data.groupby(['Team']).lose.sum().values
    table['Goals for']=season_data.groupby(['Team']).Goals_scored.sum().values
    table['Goals against']=season_data.groupby(['Team']).Goals_conceded.sum().values
    table['Goals diff']=table['Goals for']-table['Goals against']
    table['Points']=table['win']*3+table['Draw']
    table.sort_values(['Points'],inplace=True,ascending=False)
    table.reset_index(inplace=True,drop=True)
    #table.reset_index(inplace=True)
    return table

## Select Date of Choice 

In [ ]:
ref_date = "2018-01-08" #@param {type:"date"}


In [ ]:
ref_date=dt.date(dt.fromisoformat(ref_date))
show_table(ref_date)

,Team,win,Played,Draw,Loss,Goals for,Goals against,Goals diff,Points
0,Medusa Merger,6.0,9,3.0,0.0,17.0,5.0,12.0,21.0
1,Tadpole,5.0,7,2.0,0.0,10.0,4.0,6.0,17.0
2,Cartwheel,4.0,9,3.0,2.0,16.0,14.0,2.0,15.0
3,Fireworks,3.0,10,4.0,3.0,11.0,11.0,0.0,13.0
4,Milky Way,4.0,7,1.0,2.0,12.0,9.0,3.0,13.0
5,Cosmos Redshift 7,3.0,6,2.0,1.0,7.0,5.0,2.0,11.0
6,Backward,3.0,6,1.0,2.0,10.0,9.0,1.0,10.0
7,Andromeda,2.0,5,3.0,0.0,11.0,5.0,6.0,9.0
8,Antennae,2.0,7,2.0,3.0,10.0,12.0,-2.0,8.0
9,Triangulum,1.0,7,5.0,1.0,5.0,6.0,-1.0,8.0
